# multiscale model fitting for Toy3a



### start with initalizing many things

In [1]:
# imports
import os
import sys
import torch
# import pickle
import numpy as np
import matplotlib.pyplot as plt
# from tqdm.notebook import tqdm
# import time
import math

module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
        
# import torch_cae_multilevel_V4 as net
import ResNet as tnet
import utils
import training_class as tc

if torch.cuda.is_available():
    torch.cuda.set_per_process_memory_fraction(0.5, 0)

reloaded
using new ResNet thing


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# paths
data_dir = '../data/toy4a'
model_dir = '../model/toy4a'
result_dir = '../result/toy4a'

obj = tc.training_class(data_dir, model_dir, result_dir,tol=1e-4,n_inputs=4)



device =  cuda:0
i =  0
Transforming
reshape to print
i =  1
Transforming
reshape to print
i =  2
Transforming
reshape to print
i =  3
Transforming
reshape to print
i =  4
Transforming
reshape to print
i =  5
Transforming
reshape to print
dict_keys(['64', '32', '16', '8', '4', '2', '1'])
i =  0
Transforming
reshape to print
i =  1
Transforming
reshape to print
i =  2
Transforming
reshape to print
i =  3
Transforming
reshape to print
i =  4
Transforming
reshape to print
i =  5
Transforming
reshape to print
dict_keys(['64', '32', '16', '8', '4', '2', '1'])
device =  cuda:0


In [ ]:
import training_class as tc
obj, resolved = tc.train_one_step(obj, 1, verbose=True, make_new = True, start_k=1, largest_k=6)

inside train_one_timestep
Making a new model. Old one deleted. model model_L1_D2_noise0.pt
create model ../model/toy4a/model_L1_D2_noise0.pt ...
device =  cuda:0
data shape =  torch.Size([20, 501, 1, 1])
train_data  torch.Size([20, 251, 1])
data shape =  torch.Size([10, 501, 1, 1])
train_data  torch.Size([10, 251, 1])
device =  cuda:0
epoch  1000 : train_error:  0.7912312 : val_loss  1.0088267 : min_val_loss  1.0486324
epoch  2000 : train_error:  0.30586734 : val_loss  0.3893624 : min_val_loss  0.47216558
epoch  3000 : train_error:  0.023003023 : val_loss  0.030425647 : min_val_loss  0.031698838
epoch  4000 : train_error:  0.018685598 : val_loss  0.024781097 : min_val_loss  0.024910681
epoch  5000 : train_error:  0.018191535 : val_loss  0.024135401 : min_val_loss  0.024181949
epoch  6000 : train_error:  0.017678218 : val_loss  0.023351626 : min_val_loss  0.023468776
epoch  7000 : train_error:  0.017214961 : val_loss  0.0227802 : min_val_loss  0.022935048
epoch  8000 : train_error:  0.0

In [ ]:
import utils
print(obj.model_keep)
utils.plot_lowest_error(obj.model_keep[0])

In [ ]:
model = obj.model_keep[0]
y_preds, mse = model.predict_mse()
plt.plot(y_preds[0].detach().numpy())
plt.plot(model.val_data[0, ::model.step_size, 0, 0].cpu())

In [ ]:
obj, resolved = tc.train_next_step(obj, 2, verbose=False)

In [ ]:
import training_class as tc
obj, resolved = tc.train_next_step(obj, 4, verbose=False)

In [ ]:
import training_class as tc
obj, resolved = tc.train_next_step(obj, 8, verbose=True)

In [ ]:
print(obj.model_used_dict['4'])
print(obj.model_used_dict['8'])


In [ ]:
from scipy.interpolate import interp1d
def interpolate(self,current_size,model):
    predicted_this, mse = model.predict_mse()
    n_points, n_timesteps = predicted_this.shape
    interp = np.zeros(predicted_this.shape)
    mse_sum = 0
    for i in range(interp.shape[0]):
        self.model_keep[1].step_size
        x = np.arange(4,n_timesteps+4)*model.step_size#current_size
        print("step_size = ", model.step_size)
        y = predicted_this[i]
#         y = np.append()
        print(x.shape)
        print(y.shape)
        f = interp1d(x, y)
        print((min(x), max(x)))
        xnew = np.arange(min(x), max(x))#, num=41, endpoint=True)
        print(f(xnew).shape)
        plt.plot(x, y, 'o', label = "old")
        plt.plot(xnew, f(xnew), label = "new")
#         interp[i] = f(xnew)
#         plt.plot(np.arange(16,500), obj.val_dict[str(current_size)][i,16:,0,0], label = "truth")
        plt.plot(obj.val_dict[str(current_size)][i,:,0,0], label = "truth")
#         plt.semilogy(np.arange(16,500), np.abs(obj.val_dict[str(current_size)][i,16:,0,0] - f(xnew)))
        plt.legend()
#         plt.xlim([0,100])
        plt.show()
    
        mse_sum += torch.mean((obj.val_dict[str(current_size)][0,116:216,0,0] - f(xnew)[100:200])**2)
        print("mse = ", torch.mean((obj.val_dict[str(current_size)][0,116:216,0,0] - f(xnew)[100:200])**2))
        
        
    print("mse total= ", mse_sum / 10)
    return interp
    
interp = interpolate(obj,4,obj.model_keep[3])


In [ ]:
def predict_mse(self, data=None):
    n_inputs = 3
    
    if data is None:
        #use data in model if none imported
        data = self.val_data[:, ::self.step_size]
            
    n_points, n_timesteps, _, _ = data.shape
    mse_list = np.zeros(n_points)
    pred_list_all = torch.ones(data.shape[:(n_inputs-1)])*(-1)
    for num in range(n_points):
        y_pred = data[num, :n_inputs, 0, 0].float().T
        pred = y_pred
        
        for i in range(n_timesteps-n_inputs):
            y_next = self.forward(y_pred)
            pred = torch.cat((pred,y_next))
            y_next = torch.cat((y_pred[1:], y_next))
            
            
            y_pred = y_next

        mse = np.mean((pred.detach().numpy() - data[num].detach().numpy())**2)
        mse_list[num] = mse
        pred_list_all[num] = pred
    return pred_list_all, np.mean(mse_list)


model=obj.model_keep[0]
y_pred, mse = predict_mse(model)
print(mse)
print("y_pres = ", y_pred.shape)
plt.plot(y_pred[0,:].detach().numpy())
plt.plot(model.val_data[:, ::model.step_size][0,:,0,0])

In [ ]:
print("y_pres = ", y_pred.shape)
plt.plot(y_pred[0,:].detach().numpy())
plt.plot(model.val_data[:, ::model.step_size][0,:,0,0])

In [ ]:
model=obj.model_keep[1]
num = 0
data = model.val_data[:,::model.step_size]
print(data.shape)
inputs = data[num,:3,0]#, data[num,1:-2,0], data[num,2:-1,0]), axis = 1)
# old = inputs[0]
print(old)
print(inputs.shape)

print(inputs[0].float())

plt.plot(obj.val_dict['2'][0,:,0,0])
plt.plot(np.arange(3)*model.step_size, inputs[0])
plt.xlim([0,20])

In [ ]:
def predict(self, current_size):
    print(int(self.model_used_dict[str(current_size)][0,0]))
    predicted_this, mse = self.model_keep[int(self.model_used_dict[str(current_size)][0,0])].predict_mse()
    print(predicted_this.shape)
    n_points, n_timesteps = predicted_this.shape
    print(n_points)
    print(n_timesteps)
    predicted = np.zeros((n_points, n_timesteps, current_size, current_size))
    print(predicted.shape)
    for i in range(current_size):
        for j in range(current_size):
            print("int(self.model_used_dict[str(current_size)][i,j])= ", int(self.model_used_dict[str(current_size)][i,j]))
            model = self.model_keep[int(self.model_used_dict[str(current_size)][i][j])]
            predicted_this, mse = model.predict_mse()
            predicted[:,:,i,j] = predicted_this
    return predicted
predicted = predict(obj, 4)

In [ ]:
plt.imshow(predicted[0,1])
plt.colorbar()

In [ ]:
print(val_dict['1'].shape)
plt.plot(val_dict['1'][0,:,0,0], label = '1')
plt.plot(val_dict['2'][0,:,0,0], label = '2')
# plt.ylim([0,1])
mse = torch.mean((val_dict['2'][:,:,0,0] - val_dict['1'][:,:,0,0])**2)
print(mse)
plt.plot(train_dict['4'][0,:,0,0], label = '4')
plt.legend()
# plt.plot(train_dict['8'][0,:,0,0])
# plt.plot(train_dict['16'][0,:,0,0])
# plt.plot(train_dict['32'][0,:,0,0])
# plt.plot(train_dict['64'][0,:,0,0])
# plt.plot(train_dict['128'][0,:,0,0])
# plt.plot(np.cos(np.arange(500)/32*np.pi)/4+.5)

In [ ]:
import utils
print(val_dict['1'].shape)
resolved, loss, unresolved_list = utils.find_error_4(val_dict['1'][:10], models[idx_lowest], val_dict['2'][:10], plot = True,tol=0.0003)
print(loss.shape)
print(loss)



In [ ]:
plt.imshow(np.log10(loss))
plt.title("log loss after 1 iteration ")
plt.colorbar()

In [ ]:
# print(models[idx_lowest](val_dict['1']).shape)
model = models[idx_lowest]
y_preds, mse_avg = model.predict_mse()
print(mse_avg)
print(model.step_size)
# print(utils.mse(y_preds, val_dict['128'][:,::8,0,0]))
losses = np.zeros((64,64))
for i in range(64):
    print(i)
    for j in range(64):
        truth_with_step_size = val_dict['64'][:,::8,i,j]
        loss = utils.mse(y_preds, truth_with_step_size[:, :-3])
        losses[i,j] = loss

In [ ]:
print(losses)
print(np.mean(losses))
plt.imshow(losses)
plt.colorbar()

In [ ]:
plt.imshow(val_dict['2'][0,0])
plt.colorbar()

In [ ]:

unresolved_dict[str(current_size)] = torch.tensor(unresolved_list)

print(unresolved_list)

In [ ]:
current_size = 1
next_train_data = unresolved_list * train_dict[str(current_size*2)]
print(next_train_data.shape)
plt.imshow(next_train_data[0,0])
plt.colorbar()
plt.show()

In [ ]:
model_keep = [models[idx_lowest]]
model_used_dict[str(current_size)] = [[0]]

print(model_used_dict)

In [ ]:
import utils

current_size = 2
next_train_data = unresolved_list * train_dict[str(current_size)]

model_idx_list = np.ones((current_size, current_size))*(-1) #start with all -1

for i in range(current_size):
    for j in range(current_size):
        print("i = ", i, ": j = ", j)
        data_this = next_train_data[:,:,i,j]
        if (torch.min(data_this) == 0) and (torch.max(data_this) == 0):
            print("zero, no need to train")
            #don't need to do anything is model is resolved
            continue
        else:
        #see if the error is low enough on already made model
            for m, model in enumerate(model_keep):
                loss, resolved = utils.find_error_1(data_this, model,tol=0.0003)
                step_size = model.step_size
                print("model ", m, " has loss = ", loss)
                if resolved:
                    model_idx_list[i,j] = m
                    print("Resolved with loss = ", loss, ": model #", m)
                    break
                else:
                    pass
            if not resolved:
                print("not resolved, fitting new model")
                k = int(np.log2(step_size))
                #if no model good, train new model
                models, step_sizes, mse_list, idx_lowest, n_forward_list = utils.find_best_timestep(train_dict[str(current_size)][:,:,i,j], 
                                                              val_dict[str(current_size)][:,:,i,j], 
                                                              val_dict[str(current_size)][:,:,i,j], current_size,model_dir=model_dir,#make_new = True,
                                                              i=i, j=j, start_k = 2, largest_k =3)#, dont_train=False)
                
                model_keep.append(models[idx_lowest])
                model_idx_list[i,j] = len(model_keep)-1
model_used_dict[str(current_size)] = model_idx_list

In [ ]:
print(model_idx_list)
print(len(model_keep))
print(model_used_dict)

In [ ]:
import utils
tol = 0.0003
#once we have all 4 figured out, need to check the errors on the 2x2 of the 2x2s (the 4x4)
unresolved_list_big = np.ones((4,4))*(-1)
loss_big = np.ones((4,4))*(-1)
all_resolved = True
for i in range(2):
    for j in range(2):
        print(model_used_dict['2'][i][j])
        model = model_keep[int(model_used_dict['2'][i][j])]
        print(model.model_name)
        data_next = val_dict['4'][:,:, i*current_size:(i+1)*current_size, j*current_size:(j+1)*current_size]
        resolved, loss, unresolved_list = utils.find_error_4(val_dict['2'][i,j], model, data_next,plot = True, tol=tol)
        unresolved_list_big[i*current_size:(i+1)*current_size, j*current_size:(j+1)*current_size] = unresolved_list
        loss_big[i*current_size:(i+1)*current_size, j*current_size:(j+1)*current_size] = loss
        if not resolved:
            all_resolved = False
        print(loss)
        print(unresolved_list)
print("all_resolved = ", all_resolved)
print("unresolved_list_big = ", unresolved_list_big)
print("loss_big = ", loss_big)
plt.imshow(np.log10(loss_big),cmap='Greys')
plt.colorbar()
plt.title("log(loss)")

In [ ]:
#doing next refine, should be perfect here. 
import utils


current_size = 4
print("unresolved_list = ", unresolved_list)
next_train_data = torch.tensor(unresolved_list_big) * train_dict[str(current_size)]

model_idx_list = np.ones((current_size, current_size))*(-1) #start with all -1

for i in range(current_size):
    for j in range(current_size):
        print("i = ", i, ": j = ", j)
        data_this = next_train_data[:,:,i,j]
        if (torch.min(data_this) == 0) and (torch.max(data_this) == 0):
            print("zero, no need to train")
            #don't need to do anything is model is resolved
            continue
        else:
        #see if the error is low enough on already made model
            for m, model in enumerate(model_keep):
                loss, resolved = utils.find_error_1(data_this, model)
                step_size = model.step_size
                print("model ", m, " has loss = ", loss)
                if resolved:
                    model_idx_list[i,j] = m
                    print("Resolved with loss = ", loss, ": model #", m)
                    break
                else:
                    pass
            if not resolved:
                print("not resolved, fitting new model")
                k = int(np.log2(step_size))
                #if no model good, train new model
                models, step_sizes, mse_list, idx_lowest, n_forward_list = utils.find_best_timestep(train_dict[str(current_size)][:,:,i,j], 
                                                              val_dict[str(current_size)][:,:,i,j], 
                                                              val_dict[str(current_size)][:,:,i,j], current_size,model_dir=model_dir,#make_new = True,
                                                              i=i, j=j, start_k = max(0,k-1), largest_k = k+2)#, dont_train=False)
                
                model_keep.append(models[idx_lowest])
                model_idx_list[i,j] = len(model_keep)-1
model_used_dict[str(current_size)] = model_idx_list

In [ ]:
print(model_idx_list)
print(len(model_keep))
print(model_used_dict)

import utils
tol = 2.5e-2
#once we have all 4 figured out, need to check the errors on the 2x2 of the 2x2s (the 4x4)
unresolved_list_big = np.ones((current_size, current_size))*(-1)
all_resolved = True
for i in range(1):
    for j in range(1):
        print(model_used_dict[str(current_size)][i][j])
        model = model_keep[int(model_used_dict[str(current_size)][i][j])]
        print(model.model_name)
        data_next = val_dict[str(current_size)][:,:, i*current_size:(i+1)*current_size, j*current_size:(j+1)*current_size]
        print("data next shape = ", data_next.shape)
        resolved, loss, unresolved_list = utils.find_error_4(val_dict[str(current_size)][i,j], model, data_next,plot = True, tol=tol)
        unresolved_list_big[i*current_size:(i+1)*current_size, j*current_size:(j+1)*current_size] = unresolved_list
        if not resolved:
            all_resolved = False
        print(loss)
        print("unresolved_list= ", unresolved_list)
print("all_resolved = ", all_resolved)
print("unresolved_list_big = ", unresolved_list_big)

In [ ]:
import utils
model_slow = model_keep[3]
data = val_dict['2'][:,:, 1,1]
utils.find_error_1(data, model_slow, tol=2e-2, plot=True, title="Slow timescale")

model_slow = model_keep[0]
data = val_dict['1'][:,:, 0,0]
utils.find_error_1(data, model_slow, tol=2e-2, plot=True, title="step 1")


model_slow = model_keep[1]
data = val_dict['2'][:,:, 0,0]
utils.find_error_1(data, model_slow, tol=2e-2, plot=True, title="step 2 on split block")

model_fast = model_keep[6]
data = val_dict['4'][:,:, 1,1]
utils.find_error_1(data, model_fast, tol=2e-2, plot=True, title = "Fast timescale")

In [ ]:
print(step_sizes, mse_list, idx_lowest)
resolved, loss, unresolved_list = utils.find_error_4(val_dict[str(current_size)][:,:,i,j], 
                                                               models[idx_lowest], 
                                                               val_dict[str(current_size*2)][:,:, i*current_size:(i+1)*current_size, j*current_size:(j+1)*current_size])
print(loss)    

In [ ]:
model =  models[idx_lowest]
print(idx_lowest)
n_timesteps = 500
n_points = 10
dim = 1
# plt.plot(model(val_dict[str(current_size)][:,:,i,j].unsqueeze(2).unsqueeze(3))[0,:,0,0].detach().numpy(), label = "predicted")
print(val_dict[str(current_size)][:,0,i,j].unsqueeze(1).shape)
val_data_this = val_dict[str(current_size)][:,0,i,j].unsqueeze(1)
predicted = model.uni_scale_forecast(val_data_this, n_steps=n_timesteps-1)
print(predicted.shape)
predicted_reshape = predicted.reshape((  n_points, n_timesteps-1, dim,dim))
plt.plot(predicted_reshape[0,:,0], label = "predicted")
print(val_dict[str(current_size*2)][:,:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size].shape)
# plt.plot(val_dict[str(current_size*2)][:,1:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size][0,:,0,0], label = "Truth")

plt.plot(val_dict[str(current_size*2)][:,1:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size][0,:,0,1], label = "Truth")
plt.legend()

In [ ]:
plt.imshow(val_dict[str(current_size*2)][0,0])
plt.colorbar()

In [ ]:
step_size = (16+32)/2
print(step_size)
model = train_one_timestep(int(28), train_dict[str(current_size)][:,:,i,j].unsqueeze(2), 
                           val_dict[str(current_size)][:,:,i,j].unsqueeze(2), 
                           val_dict[str(current_size)][:,:,i,j].unsqueeze(2), current_size)
#                        dt = 1, n_forward = 5, noise=0, make_new = False, dont_train = True, 
#                        lr = 1e-3, max_epochs = 10000, batch_size = 50,threshold = 1e-4, 
#                        model_dir = './models/toy2',i=None, j = None):
    
#     train_dict[str(current_size)][:,:,i,j], 
#                                                               val_dict[str(current_size)][:,:,i,j], 
#                                                               val_dict[str(current_size)][:,:,i,j], current_size,model_dir=model_dir, 
#                                                               i=i, j=j, start_k = max(0,k-1), largest_k = k+2)

In [ ]:
i = 0
j = 1
resolved, loss, unresolved_list = find_error_4(val_dict[str(current_size)][:,:,i,j], 
                                                               model, 
                                                               val_dict[str(current_size*2)][:,:, i*current_size:(i+1)*current_size, j*current_size:(j+1)*current_size])
print(loss)

In [ ]:
# model =  models[idx_lowest]
print(idx_lowest)
n_timesteps = 500
n_points = 10
dim = 1
# plt.plot(model(val_dict[str(current_size)][:,:,i,j].unsqueeze(2).unsqueeze(3))[0,:,0,0].detach().numpy(), label = "predicted")
print(val_dict[str(current_size)][:,0,i,j].unsqueeze(1).shape)
val_data_this = val_dict[str(current_size)][:,0,i,j].unsqueeze(1)
predicted = model.uni_scale_forecast(val_data_this, n_steps=n_timesteps-1)
print(predicted.shape)
predicted_reshape = predicted.reshape((  n_points, n_timesteps-1, dim,dim))
plt.plot(predicted_reshape[0,:,0], label = "predicted")
print(val_dict[str(current_size*2)][:,:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size].shape)
# plt.plot(val_dict[str(current_size*2)][:,1:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size][0,:,0,0], label = "Truth")

plt.plot(val_dict[str(current_size*2)][:,1:, i*current_size:(i+1)*current_size, i*current_size:(i+1)*current_size][0,:,0,1], label = "Truth")
plt.legend()